### Imports

In [1]:
import os
import numpy as np
import pandas as pd

### Inputs

In [2]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_apmr_edge = radnet_input_df['apmr_edge'].values
radnet_input_mfsr_edge = radnet_input_df['mfsr_edge'].values
radnet_input_idg_global = radnet_input_df['idg_global'].iloc[0]

- apmr_edge (Active Prefix Message Ratio): Taxa de eficiência dos Active Prefixes no encaminhamento de mensagens através das arestas, 
medindo quão bem os campos de prefixo conseguem realizar matching probabilístico para encaminhar mensagens entre nós vizinhos.

- mfsr_edge (Message Forwarding Success Ratio): Proporção de mensagens transmitidas com sucesso através de uma aresta específica, 
considerando interferência, congestionamento e qualidade da conexão sem fio entre dois nós conectados.

- idg_global (Interest Group Delivery): Eficácia global da entrega de mensagens no modo Interest-Group addressing, 
medindo quão bem o protocolo consegue localizar e entregar mensagens aos nós que compartilham o mesmo interesse da aplicação.

In [3]:
inference_output_df = pd.read_csv("../outputs/inference_output.csv")
inference_output_confidence = inference_output_df['confidence'].values

- confidence (Confiança da Inferência do Modelo GNN): Eficácia do modelo gerado pelo GNN medido na atapa anterior.

In [4]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

### Fusion

In [5]:
weight_inference_model = 0.80
weight_apmr_edge = 0.05
weight_mfsr_edge = 0.05
weight_idg_global = 0.10

conf = (
    weight_inference_model*inference_output_confidence + 
    weight_apmr_edge*radnet_input_apmr_edge + 
    weight_mfsr_edge*radnet_input_mfsr_edge + 
    weight_idg_global*radnet_input_idg_global
)

In [6]:
tau = 0.5
cls = (conf >= tau).astype(int)

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

In [7]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[cls[i]]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-65    3.5   63     0.67   1        Bom      0.93      
-34    2.8   30     0.38   1        Bom      0.93      
-62    7.2   37     0.81   1        Bom      0.91      
-52    1.8   9      0.15   1        Bom      0.92      
-31    2.1   33     0.11   1        Bom      0.94      
-61    3.5   21     0.01   1        Bom      0.95      
-45    3.3   19     0.20   1        Bom      0.94      
-64    7.7   71     0.54   1        Bom      0.94      
-53    3.3   40     0.32   1        Bom      0.86      
-39    2.0   10     0.04   1        Bom      0.95      
-63    2.7   3      0.39   1        Bom      0.90      
-87    3.5   50     0.54   1        Bom      0.91      
-56    7.2   37     0.37   1        Bom      0.89      
-89    8.3   83     0.59   1        Bom      0.95      
-55    4.5   91     0.47   1        Bom      0.94      
-32    2.9   21     0.15   1        Bom      0.9

### Save

In [8]:
fusion_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
fusion_output.to_csv("../outputs/fusion_output.csv", index=False)

print("Fusão de Dados salva em ../outputs/fusion_output.csv")

Fusão de Dados salva em ../outputs/fusion_output.csv
